In [1]:
import pandas as pd
import pickle as pk
import numpy as np

from xgboost import XGBRegressor
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from pipeline.preprocess import PCAApplier

In [2]:
train_ls = pk.load(open("data/processed/full_train_landsat.pkl", "rb"))
test_ls = pk.load(open("data/processed/full_test_landsat.pkl", "rb"))

print(train_ls.shape)
print(test_ls.shape)

(88468, 60, 6)
(4716, 60, 6)


In [3]:
data = PCAApplier("train", method="full", from_year=2010).data
xx_col = [col for col in data if "species" not in col]
yy_col = [col for col in data if "species" in col] 

In [14]:
train_ls_rs = train_ls.reshape((train_ls.shape[0], 360))
train_ls_rs = train_ls_rs.astype(np.uint8)
print(train_ls_rs.shape)
print(train_ls_rs.dtype)

(88468, 360)
uint8


In [5]:
model = XGBRegressor(device="cuda")

In [32]:
xx = data[xx_col]
xx.head()

,lon,lat,year,geoUncertaintyInM,areaInM2,Austria,Belgium,Czech Republic,Denmark,France,...,pca_BioPr_03,pca_BioPr_04,pca_BioPr_05,pca_BioPr_06,pca_BioPr_07,pca_BioPr_08,pca_BioPr_09,pca_BioPr_10,pca_BioPr_11,pca_BioPr_12
surveyId,,,,,,,,,,,,,,,,,,,,,
212,3.099038,43.134956,2021,5.0,100.0,False,False,False,False,True,...,9326.813553,-595.659132,-3525.038112,-10287.502324,-17148.727127,-18567.991428,-12350.195384,12210.019770,5985.470575,-15245.853847
222,9.884560,56.912140,2017,10.0,79.0,False,False,False,True,False,...,-5031.244987,-2395.149252,-3072.149615,269.800322,-459.293839,-3786.485561,-2322.701337,-3440.435807,-4717.078459,-6789.021223
243,8.256020,55.637051,2019,10.0,79.0,False,False,False,True,False,...,-5107.339261,-3835.272822,-7954.768499,-3627.619198,-859.794562,6025.943181,12002.551561,-171.619099,186.347828,-1000.705264
324,-0.402590,43.505630,2018,1.0,79.0,False,False,False,False,True,...,13751.523676,13798.840851,14994.075643,6218.796852,-3192.296547,-12312.006623,-5151.568202,929.791646,15863.846036,5178.951049
333,-0.517360,45.806431,2017,1.0,79.0,False,False,False,False,True,...,4487.409542,7826.708150,5517.322160,1579.429154,-10098.465593,-6640.578609,-5447.924498,-138.361702,8829.104561,10265.399109


In [34]:
xxx = pd.DataFrame(train_ls.reshape(train_ls.shape[0], 360), dtype=np.uint8, index=xx.index)
xxx.head()

,0,1,2,3,4,5,6,7,8,9,...,350,351,352,353,354,355,356,357,358,359
surveyId,,,,,,,,,,,,,,,,,,,,,
212,7,9,7,28,18,10,8,10,5,33,...,8,37,31,16,10,13,6,34,25,13
222,11,15,6,98,43,21,14,17,6,112,...,7,99,62,30,14,18,9,104,53,27
243,6,9,3,28,11,5,9,10,3,18,...,5,24,14,7,7,8,4,16,17,11
324,23,21,10,79,56,30,50,41,22,79,...,5,96,39,15,24,21,11,72,61,35
333,18,19,8,81,57,32,38,24,12,79,...,10,83,59,32,31,26,14,78,75,52


In [37]:
for col in xx.columns:
    xxx[col] = xx[col]

xxx.head()

,0,1,2,3,4,5,6,7,8,9,...,pca_BioPr_03,pca_BioPr_04,pca_BioPr_05,pca_BioPr_06,pca_BioPr_07,pca_BioPr_08,pca_BioPr_09,pca_BioPr_10,pca_BioPr_11,pca_BioPr_12
surveyId,,,,,,,,,,,,,,,,,,,,,
212,7,9,7,28,18,10,8,10,5,33,...,9326.813553,-595.659132,-3525.038112,-10287.502324,-17148.727127,-18567.991428,-12350.195384,12210.019770,5985.470575,-15245.853847
222,11,15,6,98,43,21,14,17,6,112,...,-5031.244987,-2395.149252,-3072.149615,269.800322,-459.293839,-3786.485561,-2322.701337,-3440.435807,-4717.078459,-6789.021223
243,6,9,3,28,11,5,9,10,3,18,...,-5107.339261,-3835.272822,-7954.768499,-3627.619198,-859.794562,6025.943181,12002.551561,-171.619099,186.347828,-1000.705264
324,23,21,10,79,56,30,50,41,22,79,...,13751.523676,13798.840851,14994.075643,6218.796852,-3192.296547,-12312.006623,-5151.568202,929.791646,15863.846036,5178.951049
333,18,19,8,81,57,32,38,24,12,79,...,4487.409542,7826.708150,5517.322160,1579.429154,-10098.465593,-6640.578609,-5447.924498,-138.361702,8829.104561,10265.399109


In [38]:
model = XGBRegressor(device="cuda")
model.fit(xxx, data[yy_col])

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [52]:
test = PCAApplier("test", method="full", from_year=2010).data
test.shape

(4716, 61)

In [58]:
test_ls = pk.load(open("data/processed/full_test_landsat.pkl", "rb"))
test_ls = pd.DataFrame(test_ls.reshape(test_ls.shape[0], 360), dtype=np.uint8, index=test.index)
test_ls.shape
for col in test.columns:
    test_ls[col] = test[col]
# 
# predictions = model.predict(test_ls)
test_ls.head()

,0,1,2,3,4,5,6,7,8,9,...,pca_BioPr_03,pca_BioPr_04,pca_BioPr_05,pca_BioPr_06,pca_BioPr_07,pca_BioPr_08,pca_BioPr_09,pca_BioPr_10,pca_BioPr_11,pca_BioPr_12
surveyId,,,,,,,,,,,,,,,,,,,,,
642,13,12,7,61,50,29,12,13,5,104,...,-4631.441993,-1762.601987,-1598.451912,3000.100469,1088.359352,-740.520449,30.697130,-1645.602900,-2994.794534,-3897.420426
1792,24,23,14,66,70,48,25,24,14,68,...,-2562.633171,-976.752516,4934.437900,-3867.000392,138.701662,-4697.760490,-11259.688002,-7675.274118,-7141.835812,7581.082719
3256,226,212,200,227,35,25,111,103,89,106,...,11180.278651,15859.888162,19825.096225,15093.358904,3171.067707,-5696.140742,-2279.584152,9439.496269,11285.157340,-5605.084423
3855,78,69,52,125,55,38,87,84,70,113,...,16365.225193,21448.191759,42335.270566,48024.544176,29688.719478,32285.372406,21136.764173,32429.426932,28398.550717,-4332.174396
4889,45,43,40,90,32,18,10,11,4,51,...,-4357.885260,17.514364,-4214.491446,3172.133341,4557.747494,3674.289437,7100.918071,-536.372872,-1636.865417,-290.604878


In [59]:
predictions = model.predict(test_ls)

/home/tim/miniconda3/envs/geolife/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:40:56] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1713397688861/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [60]:
twenties = np.ones(predictions.shape[0], np.int8) * 20
print(twenties)

from helper import get_top_n
from pipeline.postprocessing import Postprocessor

Postprocessor(
    get_top_n(predictions, twenties),
    pred_indices=test.index,
    pred_cols = yy_col,
    save=True,
    uploadable=True
)

[20 20 20 ... 20 20 20]
:: [SAVED] uploadable_2024-05-06_18-41.csv
